In [112]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_squared_error

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
### Read in data (Jan & Feb)
df_jan = pd.read_parquet('/content/drive/MyDrive/mlops-zoomcamp/HW/HW1/fhv_tripdata_2021-01.parquet')
df_feb = pd.read_parquet('/content/drive/MyDrive/mlops-zoomcamp/HW/HW1/fhv_tripdata_2021-02.parquet')

Q1: Read the data for January. How many records are there?

1054112
1154112
1254112
1354112

In [18]:
len(df_jan)

1154112

Q2: What's the average trip duration in January?

15.16
19.16
24.16
29.16

In [19]:
df_jan.head(2)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009


In [120]:
df_jan["trip_dur"] = (df_jan.dropOff_datetime - df_jan.pickup_datetime).astype('timedelta64[m]')
df_jan.head(2)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,trip_dur
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.0
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.0


In [121]:
avg_during = sum(df_jan["trip_dur"]) /len(df_jan)

In [165]:
df_jan["trip_dur"].mean()

18.733049305440026

In [122]:
avg_during

18.733049305440026

In [123]:
## remove records where the duration was between 1 and 60 minutes (inclusive)
df_jan_filtered= df_jan[(df_jan["trip_dur"] <= 60) & (df_jan["trip_dur"]>=1)]

In [166]:
df_jan_filtered["trip_dur"].mean()

15.8499162370496

In [124]:
df_jan_filtered

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,trip_dur
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.0
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.0
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.0
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.0
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,NaN,71.0,None,B00037,9.0
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,8.0
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,57.0
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,16.0
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,19.0


Q3: What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.

53%
63%
73%
83%


In [125]:
df_jan_filtered['PUlocationID'] = df_jan_filtered['PUlocationID'].fillna(-1)

In [126]:
df_jan_filtered

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,trip_dur
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,NaN,None,B00009,17.0
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,NaN,None,B00009,17.0
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,None,B00037,8.0
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,None,B00037,15.0
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,None,B00037,9.0
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,8.0
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,57.0
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,16.0
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,19.0


In [127]:
hasvalue = df_jan_filtered[df_jan_filtered['PUlocationID'] != -1] 

In [128]:
len(hasvalue)

183079

In [129]:
len(hasvalue["PUlocationID"].notna())

183079

In [130]:
df_jan_replaced = hasvalue[hasvalue['DOlocationID'] != -1] 

In [131]:
len(df_jan_replaced["PUlocationID"].notna())

183079

In [132]:
len(df_jan_replaced["DOlocationID"].notna())

183079

In [72]:
len(hasvalue)

183079

In [133]:
fraction_missing = (1110873 - 183079) / 1110873

In [134]:
fraction_missing

0.8351935819846193

Q4: What's the dimensionality of this matrix? (The number of columns).

2
152
352
525
725


In [135]:
df_pianddr = df_jan_replaced[["PUlocationID", "DOlocationID"]]

In [136]:
df_pianddr['DOlocationID'].isnull().values.any()

True

In [137]:
df_pianddr['DOlocationID'] = df_pianddr['DOlocationID'].fillna(-1)

In [138]:
jan_dict = df_pianddr.to_dict('records')

In [91]:
# jan_dict

In [139]:
v = DictVectorizer(sparse=False)
X = v.fit_transform(jan_dict)
X

array([[237., 236.],
       [196., 196.],
       [ 16., 252.],
       ...,
       [ 91.,  44.],
       [171., 171.],
       [ 15.,  15.]])

In [140]:
X.shape

(183079, 2)

Q5: What's the RMSE on train?
5.52
10.52
15.52
20.52

In [141]:
y = list(df_jan_replaced["trip_dur"])

In [142]:
reg = LinearRegression().fit(X, y)

In [151]:
y_pred_jan = reg.predict(X)

In [160]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [161]:
rmse_jan = rmse(y, y_pred_jan)

In [162]:
rmse_jan

12.756418633307502

In [143]:
df_feb["trip_dur"] = (df_feb.dropOff_datetime - df_feb.pickup_datetime).astype('timedelta64[m]')


In [144]:
df_feb['DOlocationID'] = df_feb['DOlocationID'].fillna(-1)
df_feb["PUlocationID"] = df_feb['PUlocationID'].fillna(-1)

In [145]:
df_feb_test = df_feb[["PUlocationID","DOlocationID"]]

In [146]:
test_dict = df_feb_test.to_dict('records')

In [147]:

X_test = v.fit_transform(test_dict)
X_test

array([[ -1.,  -1.],
       [ 82., 173.],
       [ 56., 173.],
       ...,
       [171.,  28.],
       [252.,  16.],
       [135., 171.]])

In [148]:
y_predict = reg.predict(X_test)

In [149]:
y_true = list(df_feb["trip_dur"])

In [164]:
rmse(y_true, y_predict)

161.1257542508441